In [1]:
import sys
import argparse

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, 
    substring,
    lit,
    trim,
    replace,
    translate
)

In [2]:
execution_date = '2025-05-16'
read_path  = f'../data/3_raw_news/01_set/{execution_date}'
write_path = f'../data/4_clean_news/01_set/{execution_date}'

In [3]:
spark = (
    SparkSession
        .builder
        .appName('POC Clean data 1')
        .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/16 21:01:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.options(header=True, multiLine=True, escape='"').csv(read_path)
df.show()

+--------------------+--------------------+--------------------+
|       news_datetime|          news_title|        news_content|
+--------------------+--------------------+--------------------+
|2025-05-06T19:05:...|คำอธิบายและวิเครา...|คำอธิบายและวิเครา...|
|2025-05-06T17:53:...|การแต่งตั้งกรรมกา...|เปลี่ยนแปลงกรรมกา...|
|2025-05-06T17:38:...|งบการเงิน ไตรมาสท...|                NULL|
|2025-05-06T17:37:...|คำอธิบายและวิเครา...|คำอธิบายและวิเครา...|
|2025-05-06T17:36:...|สรุปผลการดำเนินงา...|\n               ...|
+--------------------+--------------------+--------------------+



In [5]:
df2 = df \
 .withColumn('news_datetime',  substring('news_datetime', 1, 10)) \
 .withColumn('news_content',   trim(translate('news_content', '\n', ' '))) \
 .withColumn('execution_date', lit(execution_date))

df2.show()

+-------------+--------------------+--------------------+--------------+
|news_datetime|          news_title|        news_content|execution_date|
+-------------+--------------------+--------------------+--------------+
|   2025-05-06|คำอธิบายและวิเครา...|"คำอธิบายและวิเคร...|    2025-05-16|
|  "\n2025-05-|                NULL|                NULL|    2025-05-16|
|   สิ้นสุดวาร|                NULL|                NULL|    2025-05-16|
|   2025-05-06|งบการเงิน ไตรมาสท...|                NULL|    2025-05-16|
|   2025-05-06|คำอธิบายและวิเครา...|"คำอธิบายและวิเคร...|    2025-05-16|
|  \n2025-05-0|                NULL|                NULL|    2025-05-16|
|             |                 583|            526    8|    2025-05-16|
+-------------+--------------------+--------------------+--------------+



In [6]:
df2.write.mode("overwrite").option("header", True).csv(write_path)